In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

print(sys.path)

['', '/home/caleml/partial-labels/experiments/notebooks', '/home/caleml/partial-labels', '/usr/lib/python34.zip', '/usr/lib64/python3.4', '/usr/lib64/python3.4/plat-linux', '/usr/lib64/python3.4/lib-dynload', '/home/caleml/.local/lib/python3.4/site-packages', '/usr/lib64/python3.4/site-packages', '/usr/lib64/python3.4/_import_failed', '/usr/lib/python3.4/site-packages', '/usr/lib/python3.4/site-packages/IPython/extensions', '/home/caleml/.ipython']


In [3]:
import numpy as np
import tensorflow as tf

In [4]:
from data.pascalvoc.pascalvoc import PascalVOC, NB_CLASSES

In [5]:
from model.networks.baseline import Baseline
from model import metrics

In [6]:
import sklearn
sklearn.__version__

'0.20.2'

In [13]:
from experiments.launch import parse_options_file
from config import config_utils

## val dataset

In [7]:
dataset_path = '/share/DEEPLEARNING/datasets/pascalvoc/'
dataset_val = PascalVOC(dataset_path, 2510, 'val', x_keys=['image'], y_keys=['multilabel'])
print(len(dataset_val))

1


In [8]:
x_val, y_val = dataset_val[0]

## eval

In [26]:
# partial_folder = '/home/caleml/partial_experiments/exp_20190624_1613_baseline'
partial_folder = '/home/caleml/partial_experiments/exp_20190627_1641_baseline'
model_weights = {
    # 'model_10': os.path.join(partial_folder, 'weights_10_006.h5'),
    # 'model_20': os.path.join(partial_folder, 'weights_20_020.h5'),
    # 'model_30': os.path.join(partial_folder, 'weights_30_003.h5'),
    'model_100': '/home/caleml/partial_experiments/exp_20190621_1834_baseline/weights_007.h5'
}

In [27]:
# config loading
# conf = parse_options_file(os.path.join(partial_folder, 'config.yaml'))
conf = parse_options_file('/home/caleml/partial-labels/config/baseline101.yaml')
config_utils.update_config(conf)


Loaded config

{'archi': {'classifier': 'resnet101', 'loss': 'bce', 'name': 'baseline'},
 'batch_size': 8,
 'dataset': {'name': 'pascalvoc',
             'path': '/share/DEEPLEARNING/datasets/pascalvoc/'},
 'training': {'n_epochs': 20}}



In [28]:
# eval loop

for model_name, model_path in model_weights.items():
    print('Doing model %s' % model_name)  
    model = Baseline(None, NB_CLASSES)
    model.load_weights(model_path)
    
    y_pred = model.predict(x_val)
    print('y pred example %s' % str(y_pred[0]))
    
    map_metric = metrics.MAP()
    map_2 = metrics.calculate_map(y_val[0], y_pred)
    print('map2 %s' % map_2)
    ap_scores = map_metric.compute_separated(y_val[0], y_pred)
    # ap_scores = map_metric.compute_separated(y_pred, y_val[0])
    print('ap scores for %s: %s' % (model_name, str(ap_scores)))
    map_score = sum(ap_scores) / len(ap_scores)

    print('%s: %s' % (model_name, map_score))

Doing model model_100
Outputs shape (None, 20)
Final model summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 448, 448, 3)       0         
_________________________________________________________________
cls_model (Model)            (None, 14, 14, 2048)      42658176  
_________________________________________________________________
flatten_5 (Flatten)          (None, 401408)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)                8028180   
Total params: 50,686,356
Trainable params: 50,581,012
Non-trainable params: 105,344
_________________________________________________________________
y pred example [0.05703787 0.03876966 0.06486665 0.02170656 0.06401902 0.03992372
 0.23413308 0.05507912 0.07292669 0.02245397 0.03004029 0.11245652
 0.06244062 0.03530599 0.33977216 0.0383

In [ ]:
# print(y_val)
print(type(y_val[0]))
print(y_val[0].shape)
print(y_pred_100.shape)

print(y_pred_100[-1])
print(y_pred_100[2])

print(y_pred_10[-1])
print(y_pred_10[2])

print(y_pred_20[-1])
print(y_pred_20[2])

In [20]:
print(y_val[0][0])

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
